In [19]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
DATA_DIR = '../input/faq-ques/'

In [21]:
pip install -U sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [22]:
from sentence_transformers import SentenceTransformer, util


In [23]:
data_df = pd.read_csv(DATA_DIR+'FAQs.csv')
data_df.head()

,Question,Answer
0,When was Albert Einstein born?,Albert Einstein was born on 14 March 1879.
1,Where was he born?,"He was born in Ulm, Germany."
2,When did he die?,"He died 18 April 1955 in Princeton, New Jersey..."
3,Who were his parents?,His father was Hermann Einstein and his mother...
4,Did he have any sisters and brothers?,He had one sister named Maja.


In [36]:
test_df = pd.read_csv(DATA_DIR+'FAQs_test.csv')
test_df.head()

,Question
0,What is the date of his death?
1,Did Einstein have siblings?
2,Who was his wife?
3,What was Einstein's father's name?
4,At what institutions did he study?


In [24]:
data_df.shape

(10, 2)

In [25]:
ans_dict = dict(zip(data_df.index, data_df['Answer']))

In [26]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [27]:
embeddings = model.encode(data_df['Question'].values)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
def predict(ques):
    embeded_ques = model.encode(ques)
    em_vec = np.vstack([embeded_ques]*10)
    scores = cosine_similarity(embeddings, em_vec)
    idx = np.argmax([np.mean(arr) for arr in scores])
    return ans_dict.get(idx)
    

In [35]:
#predict("What was his father's name?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'His father was Hermann Einstein and his mother was Pauline Einstein (born Koch).'

In [40]:
for idx, row in test_df.iterrows():
    print(row['Question'], predict(row['Question']))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

What is the date of his death? He died 18 April 1955 in Princeton, New Jersey, USA.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Did Einstein have siblings? Albert Einstein was born on 14 March 1879.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Who was his wife? He was married to Mileva Marić between 1903 and 1919. They had three children, Lieserl (born 1902), Hans Albert (born 1904) and Eduard (born 1910). He married Elsa Löwenthal in 1919 and they lived together until her death in 1936.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

What was Einstein's father's name? Albert Einstein was born on 14 March 1879.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

At what institutions did he study? He received his main education at the following schools: Catholic elementary school in Munich, Germany (1885-1888)Luitpold Gymnasium in Munich, Germany (1888-1894) Cantonal school in Aarau, Switzerland (1895-1896) Swiss Federal Institute of Technology in Zurich, Switzerland (1896-1900) Ph.D. from Zurich University, Switzerland (1905)
